In [1]:
from itertools import combinations
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pymongo
import nltk
from nltk.corpus import stopwords
import pymorphy2
from tqdm import tqdm_notebook
nltk.download('stopwords')
stop = stopwords.words('russian')

from news.settings import MONGO_DB, MONGO_URI

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(token):
    return morph.parse(token)[0].normal_form

In [3]:
client = pymongo.MongoClient(MONGO_URI)
db = client[MONGO_DB]
collection = db['news']

In [4]:
news = pd.DataFrame(list(collection.find()))
news.head()

,_id,text
0,5aa2affd1ca8146d0ab2cc9e,Россия может обратиться в суд ВТО из-за введен...
1,5aa2affd1ca8146d0ab2cc9f,В Пентагоне допустили возможность уничтожения ...
2,5aa2affd1ca8146d0c6e2103,Председатель Стратегического командования Воор...
3,5aa2affd1ca8146d0c6e2104,Еврокомиссия одобрила предложение предоставить...
4,5aa2affd1ca8146d0ab2cca0,Немецкий производитель одежды и обуви для спор...


In [5]:
news['text'] = news['text'].str.replace('\xa0', ' ').replace('\n', '')
news['text'] = news['text'].str.replace('\n', '')

In [6]:
tfidf = TfidfVectorizer(stop_words=stop, ngram_range=(1, 3), preprocessor=lemmatize)

In [7]:
tfidf_matrix = tfidf.fit_transform(news['text'])

In [63]:
duplicates = []
squares = np.asarray(csr_matrix.sum(csr_matrix.power(tfidf_matrix, 2), axis=1))
lengths = squares ** 0.5
for i in tqdm_notebook(range(1, tfidf_matrix.shape[0])):
    rolling_matrix = csr_matrix(vstack((tfidf_matrix[i:], tfidf_matrix[:i])))
    vec_mul = csr_matrix.sum(csr_matrix.multiply(tfidf_matrix, rolling_matrix), axis=1)
    sim = vec_mul / (lengths * np.roll(lengths, i) + 1e-8)
    indexes = np.argwhere(sim > 0.9)[:, 0]
    duplicates += list(zip(indexes, (indexes+i)%tfidf_matrix.shape[0]))

In [65]:
len(duplicates)

73802

In [60]:
duplicates[96]

(40620, 40640)

In [61]:
news.iloc[40620, 1]

'Россия, Евросоюз и Украина договорились о зимних поставках газа Киеву. Об этом сообщает вечером в пятницу, 25 сентября,  со ссылкой на информированный источник в кулуарах трехсторонней министерской  в Брюсселе.«Принципиальная договоренность по "зимнему пакету" достигнута, мы рассчитываем на скорое подписание документов», — сказал источник, не уточнив детали соглашения.Ранее 25 сентября министр энергетики Украины Владимир Демчишин , что новая цена на газ, предложенная Россией, «ближе к справедливому уровню». Более подробно комментировать возможное соглашение он отказался. В свою очередь, министр энергетики России Александр Новак рассказал, что размер скидки на газ для Украины составляет примерно 20 долларов за тысячу кубометров.24 сентября премьер-министр России Дмитрий Медведев подписал постановление, согласно которому цена на газ для Киева установлена на уровне цен для стран Евросоюза, сопредельных с Украиной. Из документа следует, что стоимость топлива составит 227,36 доллара за тыс

In [62]:
news.iloc[40640, 1]

'Трехсторонние переговоры России, ЕС и Украины начались в Брюсселе. Об этом сообщил источник в европейской делегации в кулуарах встречи, передает . Он подтвердил, что «все участники переговоров нацелены на подписание сегодня финального соглашения о ценах и условиях поставок и транзита российского газа».Министр энергетики Украины Владимир Демчишин ранее заявил, что новая цена на газ, предложенная Россией, «ближе к справедливому уровню». Более подробно комментировать возможное соглашение он отказался.В свою очередь, министр энергетики России Александр Новак рассказал, что размер скидки на газ для Украины составляет примерно 20 долларов за тысячу кубометров.24 сентября премьер-министр России Дмитрий Медведев подписал постановление, согласно которому цена на газ для Киева установлена на уровне цен для стран Евросоюза, сопредельных с Украиной. Из документа следует, что стоимость топлива составит 227,36 доллара за тысячу кубометров.Скидка на газ для Украины на четвертый квартал составила 24,